In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

if os.path.abspath('../') not in sys.path:
    sys.path.insert(0, os.path.abspath('../'))

In [2]:
economic = pd.read_excel('../data/Viecon_data.xlsx')
forest_carbon = pd.read_excel('../data/deforest_carbon.xlsx')

In [3]:
economic

,province,year,AREA OF LAND (Thous. ha),POPULATION (Thous. pers.),POPULATION DENSITY (Person/km2),At current prices (Bill. dongs),State budget revenue (Bill. dongs),State budget expenditure (Bill. dongs),Investment at current prices (Bill. dongs),Number of farms,Planted area of cereals (Thous. ha),Production of fishery (Ton),Index of industrial production (%),Retail sales of goods at current prices (Bill. dongs),Number of schools (School),Number of medical establishments (Esta.)
0,An Giang,2015,353.7,2000.9,566.0,60466.8,13141.9,17687.6,11228.7,697.0,652.8,348079.0,105.0,35835.0,547.0,191
1,An Giang,2016,353.7,1976.4,559.0,65466.9,14282.0,18002.0,11920.3,1180.0,677.1,369843.0,105.7,38093.0,542.0,192
2,An Giang,2017,353.6,1954.2,553.0,70719.7,16165.1,22522.4,13488.6,1063.0,649.4,401724.0,106.8,43129.0,536.0,3560
3,An Giang,2018,353.7,1931.0,546.0,78152.9,19303.5,24880.1,16146.9,1016.0,630.6,486804.0,108.9,47437.0,530.0,3807
4,An Giang,2019,353.7,1907.4,539.0,84939.6,20539.3,26701.5,17695.7,966.0,632.2,539909.0,109.9,51421.0,531.0,3969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,Yen Bai,2017,688.7,807.3,117.2,24917.8,14616.7,14413.5,10370.8,16.0,70.7,7497.0,106.9,12665.0,192.0,358
437,Yen Bai,2018,688.7,814.6,118.3,27590.4,16345.4,16025.7,11105.3,32.0,70.6,9017.0,104.3,13868.0,197.0,371
438,Yen Bai,2019,689.3,823.0,119.4,30530.1,18736.1,18480.1,13513.1,34.0,71.6,10485.0,110.9,15979.0,179.0,393
439,Yen Bai,2020,689.3,831.6,120.6,33414.8,20577.5,20472.8,15655.2,17.0,72.3,11634.0,108.1,16728.0,177.0,409


In [9]:
forest_carbon
# sort the data by province and year 
forest_carbon = forest_carbon.sort_values(by=['Province', 'years'])
forest_carbon

,carbon_gross_emissions,tc_loss_ha,years,Province
0,24733,229,2015,An Giang
63,110738,904,2016,An Giang
126,200469,930,2017,An Giang
189,126319,540,2018,An Giang
252,34024,174,2019,An Giang
...,...,...,...,...
188,360933,752,2017,Yen Bai
251,226560,450,2018,Yen Bai
314,12688,63,2019,Yen Bai
377,970,8,2020,Yen Bai


In [13]:
# merge the economic and forest_carbon data 
Vienam_data = pd.merge(economic, forest_carbon, how='left', left_on=['province', 'year'], right_on=['Province', 'years'])
Vienam_data.drop(['Province', 'years'], axis=1, inplace=True)

In [14]:
Vienam_data

,province,year,AREA OF LAND (Thous. ha),POPULATION (Thous. pers.),POPULATION DENSITY (Person/km2),At current prices (Bill. dongs),State budget revenue (Bill. dongs),State budget expenditure (Bill. dongs),Investment at current prices (Bill. dongs),Number of farms,Planted area of cereals (Thous. ha),Production of fishery (Ton),Index of industrial production (%),Retail sales of goods at current prices (Bill. dongs),Number of schools (School),Number of medical establishments (Esta.),carbon_gross_emissions,tc_loss_ha
0,An Giang,2015,353.7,2000.9,566.0,60466.8,13141.9,17687.6,11228.7,697.0,652.8,348079.0,105.0,35835.0,547.0,191,24733,229
1,An Giang,2016,353.7,1976.4,559.0,65466.9,14282.0,18002.0,11920.3,1180.0,677.1,369843.0,105.7,38093.0,542.0,192,110738,904
2,An Giang,2017,353.6,1954.2,553.0,70719.7,16165.1,22522.4,13488.6,1063.0,649.4,401724.0,106.8,43129.0,536.0,3560,200469,930
3,An Giang,2018,353.7,1931.0,546.0,78152.9,19303.5,24880.1,16146.9,1016.0,630.6,486804.0,108.9,47437.0,530.0,3807,126319,540
4,An Giang,2019,353.7,1907.4,539.0,84939.6,20539.3,26701.5,17695.7,966.0,632.2,539909.0,109.9,51421.0,531.0,3969,34024,174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,Yen Bai,2017,688.7,807.3,117.2,24917.8,14616.7,14413.5,10370.8,16.0,70.7,7497.0,106.9,12665.0,192.0,358,360933,752
437,Yen Bai,2018,688.7,814.6,118.3,27590.4,16345.4,16025.7,11105.3,32.0,70.6,9017.0,104.3,13868.0,197.0,371,226560,450
438,Yen Bai,2019,689.3,823.0,119.4,30530.1,18736.1,18480.1,13513.1,34.0,71.6,10485.0,110.9,15979.0,179.0,393,12688,63
439,Yen Bai,2020,689.3,831.6,120.6,33414.8,20577.5,20472.8,15655.2,17.0,72.3,11634.0,108.1,16728.0,177.0,409,970,8


### Dataset Description
The dataset provides comprehensive information on various socio-economic and environmental factors across 63 provinces in Vietnam by years (from 2015-2021)
1. ```Province (string):``` Represents information on the 63 provinces and municipalities in Vietnam.
2. ```Year:``` Indicates the year in which the data was collected.
3. ```AREA OF LAND (int):```Represents the total natural land area of each province per year (in thousand hectares).
4. ```POPULATION (float):``` Signifies the total population within the geographical area (in thousand persons).
5. ```POPULATION DENSITY (float):```Measures the population density in persons per square kilometer.
6. ```GROSS REGIONAL DOMESTIC PRODUCT (GRDP) (float):```Reflects the total value of goods and services produced in the economic region each year (in billion Vietnamese dong).
7. ```State Budget Revenue (float):```Represents the total revenue collected from taxes, fees, etc., by the province in the corresponding year (in billion Vietnamese dong).
8. ```State Budget Expenditure (float):```Indicates the total amount spent by the province on programs, projects, or other activities (in billion Vietnamese dong).
9. ```Investment at Current Prices (float):```Denotes the total capital invested by businesses, organizations, or the government in a specific region, measured at current values (in billion Vietnamese dong).
10. ```Number of Farms (int):```Represents the total count of farms in the region.
11. ```Planted Area of Cereals (float):```Indicates the total land area used for cultivating cereals such as wheat or other grains (in thousand hectares).
12. ```Production of Fishery (float):```Signifies the total quantity of fisheries produced in the region (in tons).
13. ```Index of Industrial Production (float):```Measures the growth or decline of industrial production activities in the region, expressed as a percentage.
14. ```Retail Sales of Goods at Current Prices (float):```Represents the total value of goods sold at current prices in retail activities within the region (in billion Vietnamese dong).
15. ```Number of Schools (int):``` the total number of educational institutions in the region.
16. ```Number of Medical Establishments (int):``` Represents the total count of medical centers in the region.
17. ```Carbon Gross Emissions(int):```Indicates the total amount of carbon emissions released into the environment in the region (in Megaton CO2e).
18. ```tc_los_ha - Tropical Forest Loss per Hectare (Target Variable) (int):```Represents the amount of tropical forest loss per hectare in the region.

In [16]:
Vienam_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 441 entries, 0 to 440
Data columns (total 18 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   province                                               441 non-null    object 
 1   year                                                   441 non-null    int64  
 2   AREA OF LAND (Thous. ha)                               422 non-null    float64
 3   POPULATION (Thous. pers.)                              441 non-null    float64
 4   POPULATION DENSITY (Person/km2)                        425 non-null    float64
 5   At current prices (Bill. dongs)                        440 non-null    float64
 6   State budget revenue (Bill. dongs)                     438 non-null    float64
 7   State budget expenditure (Bill. dongs)                 438 non-null    float64
 8   Investment at current prices (Bill. dongs)        

In [17]:
# check for Nan values on each column 
Vienam_data.isna().sum()

province                                                  0
year                                                      0
AREA OF LAND (Thous. ha)                                 19
POPULATION (Thous. pers.)                                 0
POPULATION DENSITY (Person/km2)                          16
At current prices (Bill. dongs)                           1
State budget revenue (Bill. dongs)                        3
State budget expenditure (Bill. dongs)                    3
Investment at current prices (Bill. dongs)                0
Number of farms                                           4
Planted area of cereals (Thous. ha)                       0
Production of fishery (Ton)                               0
Index of industrial production (%)                        0
Retail sales of goods at current prices (Bill. dongs)    14
Number of schools (School)                               40
Number of medical establishments (Esta.)                  0
carbon_gross_emissions                  